In [1]:
import polars as pl

In [ ]:
path1 = ""
path2 = ""
df1 = pl.read_csv(path1)
df1.head()

index,exercise
i64,str
1,"""сед"""
2,"""квадрицепс тренажер"""
3,"""жим лежа широким хватом"""
4,"""жим лежа широким хватом 2сек"""
5,"""жим гантелей лежа"""


In [5]:
df2 = pl.read_csv(path2, try_parse_dates=True, schema={"exercise":pl.Int64, "reps":pl.String, "date": pl.Date}, ignore_errors=True, truncate_ragged_lines=True)
df2.head()

exercise,reps,date
i64,str,date
1,"""6""",2019-09-30
2,"""8""",2019-09-30
3,"""5""",2019-09-30
4,"""2""",2019-09-30
5,"""6""",2019-09-30


In [6]:
df3 = df1.join(left_on="index", right_on="exercise", other=df2).select("exercise","reps", "date")

In [7]:
df4 = df3.select("date").unique().sort(by="date").with_columns(
    pl.col("date").shift().alias("prev"),
    pl.col("date").shift(-1).alias("next")
)
df4.head()

date,prev,next
date,date,date
2019-09-30,null,2019-10-01
2019-10-01,2019-09-30,2019-10-02
2019-10-02,2019-10-01,2019-10-04
2019-10-04,2019-10-02,2019-10-07
2019-10-07,2019-10-04,2019-10-08


In [8]:
df5 = df3.join(on="date", other=df4).with_row_index()
df5

index,exercise,reps,date,prev,next
u32,str,str,date,date,date
0,"""сед""","""6""",2019-09-30,null,2019-10-01
1,"""квадрицепс тренажер""","""8""",2019-09-30,null,2019-10-01
2,"""жим лежа широким хватом""","""5""",2019-09-30,null,2019-10-01
3,"""жим лежа широким хватом 2сек""","""2""",2019-09-30,null,2019-10-01
4,"""жим гантелей лежа""","""6""",2019-09-30,null,2019-10-01
…,…,…,…,…,…
1731,"""тяга сумо вис""","""4""",2021-05-21,2021-05-19,null
1732,"""жим лежа узким хватом""","""3""",2021-05-21,2021-05-19,null
1733,"""жим лежа узким хватом 2 сек""","""1""",2021-05-21,2021-05-19,null


In [11]:
df5.write_csv(file="table.csv")

In [12]:
df5

index,exercise,reps,date,prev,next
u32,str,str,date,date,date
0,"""сед""","""6""",2019-09-30,null,2019-10-01
1,"""квадрицепс тренажер""","""8""",2019-09-30,null,2019-10-01
2,"""жим лежа широким хватом""","""5""",2019-09-30,null,2019-10-01
3,"""жим лежа широким хватом 2сек""","""2""",2019-09-30,null,2019-10-01
4,"""жим гантелей лежа""","""6""",2019-09-30,null,2019-10-01
…,…,…,…,…,…
1731,"""тяга сумо вис""","""4""",2021-05-21,2021-05-19,null
1732,"""жим лежа узким хватом""","""3""",2021-05-21,2021-05-19,null
1733,"""жим лежа узким хватом 2 сек""","""1""",2021-05-21,2021-05-19,null
